In [4]:
from os import listdir
from os.path import join as pjoin
from os.path import basename, isfile

import numpy as np
import pandas as pd
from scipy.io import loadmat

# Data conversion

Converting .mat files into dataframes and then csv files

In [35]:
def convert_to_csv(file):
    mat = loadmat(file)
    df = pd.DataFrame()
    df['leftP'] = mat['leftP'][0]
    df['rightP'] = mat['rightP'][0]
    df['trial_response_side'] = mat['trialresponseside'][0]
    df['trial_reward'] = mat['trialreward'][0]
    return df

def convert_timing_to_csv(df, file):
    mat = loadmat(file)
    # save angle data
    angle = mat['anglev'][0]
    time = np.arange(start=1, stop=len(mat['anglev'][0]) + 1) / 20000

    # reduce sample rate to 200 HZ for more efficient processing
    angle = angle[0::100]
    time = time[0::100]
    angular_velocity = np.array([angle, time]).T
    np.save(pjoin('behaviour_data', 'npy', 'wheel_velocity', basename(file).split('.')[0] + '.npy'), angular_velocity)
    df['cue_time'] = mat['cuetimes'][0]
    return df

In [37]:
# load .mat file
for file in listdir(pjoin('behaviour_data', 'mat', 'task_info')):
    path = pjoin('behaviour_data', 'mat', 'task_info', file)
    df = convert_to_csv(path) 
    timing_path = pjoin('behaviour_data', 'mat', 'timing_info', file)
    df = convert_timing_to_csv(df, timing_path)
    df.to_csv(pjoin('behaviour_data', 'csv', 'task_info', file.split('.')[0] + '.csv'), index=False)

## convert spike times

In [10]:
for dir in listdir('spike_times'):
    file = pjoin('spike_times', dir, 'spts.mat')
    if not isfile(file):
        print(dir)
        continue
    mat = loadmat(file)
    print(np.array(mat['pfcspts'][0][0]).flatten())
    pfc = [np.array(i).flatten() for i in mat['pfcspts'][0]]
    str = [np.array(i).flatten() for i in mat['strspts'][0]]
    np.save(pjoin('spike_times', dir, 'pfc.npy'), pfc)
    np.save(pjoin('spike_times', dir, 'str.npy'), str)

[   6.6482     9.35215   11.07985 ... 1997.9525  1998.39515 1998.9105 ]
AKED0120210722
[2.73000000e-01 3.36250000e-01 1.33170000e+00 ... 2.20585940e+03
 2.20693410e+03 2.20785565e+03]
[6.29950000e-01 7.49800000e-01 1.95315000e+00 ... 2.13001035e+03
 2.13015870e+03 2.13022340e+03]
[6.35100000e-01 7.63900000e-01 2.23920000e+00 ... 1.82940595e+03
 1.82968515e+03 1.83013695e+03]
[4.60150000e-01 8.85600000e-01 2.63515000e+00 ... 1.61070215e+03
 1.61117910e+03 1.61168160e+03]
[8.01750000e-01 1.08900000e+00 1.95030000e+00 ... 1.60388040e+03
 1.60390110e+03 1.61127915e+03]
[3.45500000e-02 3.64000000e-02 5.98000000e-02 ... 1.65088315e+03
 1.65096610e+03 1.65246720e+03]


c:\Python37\lib\site-packages\numpy\lib\npyio.py:528: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  arr = np.asanyarray(arr)


[   6.6482     9.35215   11.07985 ... 1997.9525  1998.39515 1998.9105 ]
[   3.2319     5.5926     6.90315 ... 1619.03595 1619.2891  1619.30505]
[7.55000000e-03 4.94000000e-02 1.58700000e-01 ... 1.50738115e+03
 1.50797485e+03 1.50799260e+03]
[3.65900000e-01 6.64950000e-01 7.05650000e-01 ... 1.86217630e+03
 1.86222435e+03 1.86231970e+03]
[   2.8518     2.8914     2.938   ... 2037.7342  2041.12815 2041.21815]
[4.35500000e-02 2.72430000e+00 2.74830000e+00 ... 2.70233000e+03
 2.70251825e+03 2.70290910e+03]
[  26.8351    45.6177    52.7854  ... 3372.76635 3372.9169  3373.2091 ]
[1.14935000e+00 1.22525000e+00 1.72260000e+00 3.29200000e+00
 9.47345000e+00 1.09227500e+01 1.09325000e+01 1.11256000e+01
 1.11559500e+01 1.18340500e+01 2.23107500e+01 2.43778000e+01
 2.44433000e+01 2.44517000e+01 2.44688500e+01 2.44797000e+01
 2.44874000e+01 2.44963000e+01 2.45059000e+01 2.46149500e+01
 2.46359500e+01 2.46534500e+01 2.49387500e+01 2.49732500e+01
 2.54533000e+01 2.54672500e+01 2.54819000e+01 2.5493350

# Convert Wei's summary table



In [3]:
from os.path import join as pjoin, isdir, basename, isfile
from os import listdir, mkdir
import numpy as np
from glob import glob
from tqdm import tqdm 
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

data = pd.read_excel('PPD_RTW_summary_table_Wei_06may2022_V2.xlsx')

# rename the sessions to match the current format
# from index_yy.mm.dd to indexyyyymmdd
data['session'] = data['session'].apply(lambda x: x.replace('.', ''))
data['session'] = data['session'].apply(lambda x: x.replace('_', '20'))

# lower case the cell location + (cell ID -1)
data['cell_name'] = data['Cell location'].apply(lambda x: x.lower()) + '_' + (data['Cell ID'] - 1).astype(str)
# remove the cell location and cell ID columns
data = data.drop(columns=['Cell location', 'Cell ID'])

# save the data in csv format
data.to_csv('delta_P_correlation.csv', index=False)